In [17]:
from pathlib import Path
import os
import sys

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
from ba_data_paths import ba_data
import pandas as pd

sys.path.append('../../src')
from geoai_retail import utils
from geoai_retail import proximity_local as proximity
from geoai_retail import analysis

In [95]:
new_loc_x, new_loc_y = -122.2753982, 47.8247231
close_loc_id = 409083547

data_dir = Path(os.path.abspath('../../data'))
interim_dir = data_dir/'interim'
raw_dir = data_dir/'raw'
raw_gdb = raw_dir/'raw.gdb'
                
nearest_stores_csv = interim_dir/'closest_store.csv'
nearest_comp_csv = interim_dir/'closest_competition.csv'

origin_fc = raw_gdb/'sea_block_group'
origin_id_fld = 'ID'

stores_fc = raw_gdb/'sea_ace'
stores_id_fld = 'LOCNUM'

comp_fc = raw_gdb/'sea_ace_comp'
comp_id_fld = 'LOCNUM'

In [86]:
stores_df = GeoAccessor.from_featureclass(str(stores_fc))
stores_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,SALESVOL,HDBRCH,ULTNUM,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,NEAR_FID,NEAR_DIST,SHAPE
0,1,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,525104,2416.0,2,002448694,15.0,1,,6,PointAddress,M,100.0000,INFOGROUP,0,3,0.000021,"{""x"": -122.29909999999995, ""y"": 47.05410000000..."
1,2,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,TACOMA,WA,Washington,98498,6154,44413005,525104,1450.0,2,002448694,9.0,1,,3,StreetAddress,M,96.3594,INFOGROUP,0,35,0.000010,"{""x"": -122.54489999999998, ""y"": 47.18000000000..."
2,3,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,525104,6120.0,2,002448694,38.0,1,,7,StreetAddress,M,100.0000,INFOGROUP,0,41,0.000019,"{""x"": -122.57929999999999, ""y"": 47.30130000000..."
3,4,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,525104,2577.0,2,002448694,16.0,1,,5,PointAddress,M,100.0000,INFOGROUP,0,96,55.586255,"{""x"": -122.43459999999999, ""y"": 47.08740000000..."
4,5,721714069,ACE HARDWARE,112TH ST S,PARKLAND,WA,Washington,98444,5711,44413005,525104,967.0,2,002448694,8.0,,,3,PointAddress,M,95.4063,INFOGROUP,0,98,0.000010,"{""x"": -122.43199999999996, ""y"": 47.15610000000..."


In [87]:
stores_df[stores_df['LOCNUM'] == str(close_loc_id)]

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,SALESVOL,HDBRCH,ULTNUM,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,NEAR_FID,NEAR_DIST,SHAPE
11,12,409083547,ACE HARDWARE-DISTR,RUSSELL RD,KENT,WA,Washington,98032,4897,44413005,525104,700.0,2,002448694,3.0,1,,2,PointAddress,M,100.0,INFOGROUP,0,278,0.000024,"{""x"": -122.26919999999996, ""y"": 47.39510000000..."


In [88]:
nearest_stores_df = pd.read_csv(str(nearest_stores_csv), index_col=0)
nearest_stores_df.head()

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06
0,530530701003,677129595,43.576384,38.513638,371889957,46.482533,42.526899,427271369,52.472263,44.462895,421027779,55.156955,44.748810,721714069,61.428125,50.456415,216082099,58.905164,54.642485
1,530530714071,371889957,9.405840,6.295963,216082099,13.296163,10.944308,421027779,16.006815,12.366455,721714069,20.814301,15.290477,677129595,29.198017,18.116390,460556608,35.087418,25.237232
2,530530714072,371889957,10.003444,6.125873,216082099,11.536480,8.195827,421027779,18.268181,13.177555,721714069,22.034957,15.623036,677129595,29.203660,20.609116,460556608,34.472733,25.286682
3,530530714073,216082099,12.308627,7.540147,371889957,13.269904,8.468519,421027779,20.223521,13.544703,721714069,23.035677,15.164948,677129595,31.159000,20.976263,460556608,35.244881,24.631002
4,530530714112,216082099,5.292562,3.556479,371889957,12.285190,9.615196,721714069,16.478653,11.395147,421027779,19.885017,15.617594,460556608,28.228815,20.647334,677129595,30.820496,23.049155


In [89]:
dest_id_cols = [c for c in nearest_stores_df if c.startswith('destination_id')]
dest_id_cols

['destination_id_01',
 'destination_id_02',
 'destination_id_03',
 'destination_id_04',
 'destination_id_05',
 'destination_id_06']

In [94]:
store_id_arr = pd.concat([nearest_stores_df[nearest_stores_df[dest_id_col] == close_loc_id]['origin_id'] 
                          for dest_id_col in dest_id_cols]).unique()
print (f'Origin Count (Block Groups) - {len(store_id_arr):,}')
store_id_arr[:20]

Origin Count (Block Groups) - 529


array([530330290011, 530330300031, 530330300032, 530330300034,
       530330283003, 530330288023, 530330289021, 530330289022,
       530330289023, 530330289024, 530330289025, 530330290031,
       530330290032, 530330290033, 530330290034, 530330290035,
       530330290041, 530330290042, 530330291011, 530330291012],
      dtype=int64)

In [91]:
origin_df = GeoAccessor.from_featureclass(str(origin_fc))
origin_df.set_index('OBJECTID', inplace=True)
origin_df.head()

,ID,NAME,SHAPE
OBJECTID,,,
1,530530701003,530530701.003,"{""rings"": [[[-122.06627999999995, 47.076520000..."
2,530530714071,530530714.071,"{""rings"": [[[-122.34031999999996, 47.071510000..."
3,530530714072,530530714.072,"{""rings"": [[[-122.35767999999996, 47.067370000..."
4,530530714073,530530714.073,"{""rings"": [[[-122.36847999999998, 47.067630000..."
5,530530714112,530530714.112,"{""rings"": [[[-122.41108999999994, 47.071690000..."


In [92]:
aoi_origin_df = origin_df[origin_df['ID'].apply(lambda val: int(val) in store_id_arr)].copy()
aoi_origin_df.head()

,ID,NAME,SHAPE
OBJECTID,,,
37,530530701001,530530701.001,"{""rings"": [[[-121.37943999999999, 47.086890000..."
48,530330265003,530330265.003,"{""rings"": [[[-122.32663599999995, 47.510322000..."
50,530330267002,530330267.002,"{""rings"": [[[-122.35516999999999, 47.502780000..."
54,530330268012,530330268.012,"{""rings"": [[[-122.33366999999998, 47.503140000..."
57,530330268021,530330268.021,"{""rings"": [[[-122.33903999999995, 47.499400001..."


In [93]:
aoi_origin_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

In [98]:
store_exclude_df = stores_df[stores_df['LOCNUM'] != str(close_loc_id)]
closest_store_df = proximity.closest_dataframe_from_origins_destinations(aoi_origin_df, origin_id_fld, store_exclude_df, 
                                                                         stores_id_fld, 
                                                                         network_dataset=ba_data.usa_network_dataset)
closest_store_df.head()

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04
0,530530701001,427271369,113.349962,81.061899,677129595,119.748479,85.531183,688834225,123.815082,91.717213,371889957,125.265106,93.243626
1,530330265003,402895226,7.747798,4.760333,700121193,13.882547,9.391289,425477834,18.620910,12.482090,426576447,18.620910,12.482090
2,530330267002,402895226,10.116710,4.611057,700121193,15.825993,9.755615,425477834,25.351087,15.046200,426576447,25.351087,15.046200
3,530330268012,402895226,7.603171,3.968376,700121193,13.499632,8.976663,425477834,20.947284,13.148341,426576447,20.947284,13.148341
4,530330268021,402895226,9.487449,4.508387,700121193,15.196732,9.652945,425477834,23.037021,13.597019,426576447,23.037021,13.597019


In [3]:
new_loc_geom = Geometry({'x': new_loc_x, 'y': new_loc_y, 'spatialReference': {'wkid': 4326}})

In [100]:
add_new_df = analysis.get_add_new_closest_dataframe(origin_df, origin_id_fld, stores_df, stores_id_fld, nearest_stores_csv, 
                                                    new_loc_geom)
add_new_df.head()

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06
0,530330002003,714937939,9.280268,3.691418,251373601,18.596257,9.161684,1,19.511233,13.117512,439724905,20.533923,13.337857,425477834,26.330957,13.390725,426576447,26.330957,13.390725
1,530330002005,714937939,6.118568,3.515287,251373601,17.887850,9.196999,1,18.802827,13.152826,425477834,23.169257,13.214595,426576447,23.169257,13.214595,439724905,19.825516,13.373172
2,530330003001,714937939,11.116407,4.758156,251373601,13.910432,9.382446,439724905,17.998856,11.418571,1,15.279377,12.854098,425477834,23.232554,13.973721,426576447,23.232554,13.973721
3,530330003002,714937939,13.259484,5.367951,251373601,16.034312,9.568386,439724905,17.713962,10.593401,1,17.403257,13.040039,718542034,22.012485,13.502292,174746248,28.975059,13.515690
4,530330004011,714937939,17.492839,7.157950,251373601,17.440866,10.926844,439724905,19.222656,11.448436,174746248,24.422088,12.627555,718542034,23.733746,14.085724,1,18.809811,14.398496
